In [5]:
import glob, os, sys; sys.path.append('../src')
from datetime import datetime

from langdetect import detect
from tabulate import tabulate 
# data wrangling
import numpy as np
import pandas as pd
import re
import ast
import itertools
import wordninja
from dateutil import parser
import wordninja
import string

from tqdm.auto import tqdm
tqdm.pandas()

'''import helper functions'''
import clean as clean
import extract_attributes as ex

'''multiprocessing'''
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

ModuleNotFoundError: No module named 'spacy'

In [ ]:
# locally
data = pd.read_csv("../data/undp_jobs.csv") 

In [ ]:
# from Zenodo
# df = pd.read_csv('https://zenodo.org/record/6589661/files/undp_jobs.csv?download=1')

In [ ]:
df = data.T
df.rename(columns={0: 'content'}, inplace=True)
df.shape

In [ ]:
df["content"]=df["content"].str.replace("{'content': ","").str.replace("}","")

In [ ]:
# delete empty rows
df = df[df['content']!="[]"]
df.shape

In [ ]:
#force column to type list
df['content_list'] = df['content'].parallel_apply(lambda l: ast.literal_eval(l))

In [ ]:
# Remove special characters, line separation, URLs and enumerations
df['content_list'] = df['content_list'].parallel_apply(lambda l: clean.clean_list(l))

In [ ]:
# Binary column whether or not a posting follows the structure (background, competencies, sills and experiences, ...)
template_structure = ['background',\
                      'duties and responsibilities',\
                      'competencies',\
                      'required skills and experience']
for section in template_structure:
    df['has_' + section.replace(' ','_')] = df['content_list'].parallel_apply(lambda l: ex.has_attribute(l, section))

In [ ]:
# Assess number of job postings not following the given structure
df[(df['has_background']==0)|\
           (df['has_duties_and_responsibilities']==0)|\
           (df['has_competencies']==0)|\
           (df['has_required_skills_and_experience']==0)]['content'].shape

In [ ]:
# Remove job postings not following the given structure and corresponding binary columns
df = df[(df['has_background']!=0)&\
           (df['has_duties_and_responsibilities']!=0)&\
           (df['has_competencies']!=0)&\
           (df['has_required_skills_and_experience']!=0)]

df.drop(['has_background',\
                  'has_duties_and_responsibilities',\
                  'has_competencies',\
                  'has_required_skills_and_experience'], axis=1, inplace=True)

print('Rows following job posting template structure: ', df.shape[0])

In [ ]:
# Extract string list of meta attributes such as application deadline, job title, ...
df['meta_atributes'] = df['content_list'].parallel_apply(lambda l: ex.get_meta_attributes(l))

# Extract job posting components following given template
df['background'] = df['content_list'].parallel_apply(lambda l: ex.get_background(l))
df['duties_and_responsibilities'] = df['content_list'].parallel_apply(lambda l: ex.get_duties_and_responsibilities(l))
df['competencies'] = df['content_list'].parallel_apply(lambda l: ex.get_competencies(l))
df['required_skills_and_experience'] = df['content_list'].parallel_apply(lambda l: ex.get_required_skills(l))

In [ ]:
meta_attributes = ['location', 'application deadline', 'time left', 'additional category',\
                  'type of contract','post level','languages required','starting date',\
                   'duration of initial contract','expected duration of assignment']

# Extract meta attributes in seperate columns
df['title'] = df['meta_atributes'].apply(lambda l: l[0])
for m_attr in meta_attributes:
    df[m_attr.replace(' ', '_')] = df['meta_atributes'].parallel_apply(lambda l: extract_meta_attribute(m_attr,l))

In [ ]:
# Consolidate column names with names from UNDP previous work
df.rename(columns={'duration_of_initial_contract':'duration_contract',\
                            'expected_duration_of_assignment':'duration_assignment',\
                            'required_skills_and_experience':'skills_experiences',\
                            'duties_and_responsibilities':'duties_responsibilities'}, inplace=True)

In [ ]:
# parse dates
df['application_deadline'] = df['application_deadline'].apply(lambda s: parser.parse(s[:9]))

In [ ]:
# extract year from date
df['year'] = df['application_deadline'].apply(lambda l: str(l.year))

In [ ]:
columns = ['content','title','location','year','time_left','additional_category','type_of_contract','post_level','languages_required','starting_date','duration_contract','duration_assignment',\
           'background','duties_responsibilities','competencies','skills_experiences']

df_jobs = df[columns]

In [ ]:
# dropping vacancies with missing data in key columns
print(f'Shape before: {df_jobs.shape}')

df_jobs.dropna(subset = ['title', 'location', 'year', 'background'], how = 'any', inplace = True)
print(f'Shape after : {df_jobs.shape}')

# dropping duplicated vacancies
to_keep = ['title', 'location', 'year', 'background', 'duties_responsibilities']
df_jobs.drop_duplicates(subset = to_keep, inplace = True)
print(f'Shape after : {df_jobs.shape}')

# generating unique vacancy ids
df_jobs.insert(0, 'job_id', df_jobs[to_keep].fillna('').apply(lambda x: ' '.join(list(x)), axis = 1))
df_jobs['job_id'] = df_jobs['job_id'].apply(lambda x: clean.generate_key(x, prefix = 'JOB'))
assert df_jobs.duplicated(subset = ['job_id']).sum() == 0, 'Non-unique id'
print(f'Shape after : {df_jobs.shape}')

# percentage not missing
display(df_jobs.notna().sum().divide(len(df_jobs) / 100).round(2).astype(str).add('%'))

# Further Cleaning and Processing

In [ ]:
# detect language 
df_jobs['language'] = [extract_language(x) for x in tqdm(df_jobs['background'])]

Now we should translate the non-english postings....

In [ ]:
# Detect and split joint words  (legacy from raw data extraction), the library used here is only defined for the English language
for field in ['background', 'duties_responsibilities', 'competencies', 'skills_experiences']:
    print('Prcessing field: ', field)
    df_jobs[field] = [clean.split_joint_words(x) for x in tqdm(df_jobs[field])]

# Locations

In [ ]:
# dependency file with locations
df_locations = pd.read_excel('../src/world-countries.xlsx', engine='openpyxl')
df_locations['country'] = df_locations['country'].str.lower()
print(f'Shape: {df_locations.shape}')
display(df_locations.head())

country2iso = dict(df_locations[['country', 'country_iso']].values)
country2region = dict(df_locations[['country_iso', 'region']].values)

In [ ]:
print('Unique count before:', df_jobs['location'].nunique())
display(df_jobs['location'].value_counts(dropna = False).head(20))

df_jobs.insert(4, 'country', df_jobs['location'].apply(lambda x: clean.clean_location(x, country2iso)))
print('Unique count after :', df_jobs['country'].nunique())
display(df_jobs['country'].value_counts(dropna = False).head(20))

df_jobs.insert(5, 'region', df_jobs['country'].apply(lambda x: clean.get_region(x, country2region)))
display(df_jobs['region'].value_counts(dropna = False))

In [ ]:
df_jobs['year'].value_counts(dropna = False).sort_index()

In [ ]:
df_jobs = df_jobs.drop('additional_category', 1)

In [3]:
print('Unique count before:', df_jobs['type_of_contract'].nunique())
display(df_jobs['type_of_contract'].value_counts(dropna = False).head(20))

df_jobs['type_of_contract'] = df_jobs['type_of_contract'].apply(clean.clean_contracts)
print('Unique count after :', df_jobs['type_of_contract'].nunique())
display(df_jobs['type_of_contract'].value_counts(dropna = False))

Unique count before: 19


individual contract                                                                         47401
service contract                                                                            18961
fta local                                                                                    7853
fta international                                                                            6607
internship                                                                                   5256
other                                                                                        1934
ald international                                                                            1447
unv                                                                                          1285
ta international                                                                             1174
200 series                                                                                    895
ta local            

Unique count after : 18


Individual Contract    47401
Service Contract       18961
FTA Local               7853
FTA International       6607
Internship              5256
Other                   1934
ALD International       1447
UNV                     1285
TA International        1174
200 Series               895
TA local                 807
100 Series               773
IPSA                     421
ALD Local                173
NPSA                     145
NPSA short-term           85
IPSA short-term           69
Unspecified                9
Name: type_of_contract, dtype: int64

In [4]:
df_jobs['post_level'] = df_jobs['post_level'].parallel_apply(clean.clean_posts)
print('Unique count after :', df_jobs['post_level'].nunique())
display(df_jobs['post_level'].value_counts(dropna = False))

Unique count after : 30


International Consultant    23179
National Consultant         20823
SB/SC/GS-3                   5766
SB/SC/GS-4                   5431
Intern                       5275
Other                        4917
Unspecified                  3898
SB/SC/GS-5                   3657
SB/SC/GS-6                   2980
P-4                          2778
P-3                          2415
SB/SC/GS-2                   1806
NO-B                         1569
P-5                          1565
SB/SC/GS-7                   1484
UNV                          1280
NO-C                         1161
SB/SC/GS-8                    902
P-2                           831
NO-A                          806
SB/SC/GS-9                    712
SB/SC/GS-1                    506
IPSA                          490
SB/SC/GS-10                   284
NPSA                          230
D-1                           207
D-2                           146
NO-D                          105
SB/SC/GS-11                    81
P-1           

In [5]:
mapping = {'D-1': 'International Professional',
           'D-2': 'International Professional',
           'Intern': 'Intern',
           'International Consultant': 'International Consultant',
           'NO-A': 'National Professional',
           'NO-B': 'National Professional',
           'NO-C': 'National Professional',
           'NO-D': 'National Professional',
           'National Consultant': 'National Consultant',
           'P-1': 'International Professional',
           'P-2': 'International Professional',
           'P-3': 'International Professional',
           'P-4': 'International Professional',
           'P-5': 'International Professional',
           'SB/SC/GS-1': 'General Service',
           'SB/SC/GS-2': 'General Service',
           'SB/SC/GS-3': 'General Service',
           'SB/SC/GS-4': 'General Service',
           'SB/SC/GS-5': 'General Service',
           'SB/SC/GS-6': 'General Service',
           'SB/SC/GS-7': 'General Service',
           'SB/SC/GS-8': 'General Service',
           'SB/SC/GS-9': 'General Service',
           'SB/SC/GS-10': 'General Service',
           'SB/SC/GS-11': 'General Service',
           'UNV': 'UNV',
           'IPSA': 'IPSA',
           'NPSA': 'NPSA',
           'Other': 'Other',
           'Unspecified': 'Unspecified'}

In [6]:
df_jobs.insert(12, 'staff_category', df_jobs['post_level'].replace(mapping).tolist())
display(df_jobs['staff_category'].value_counts(dropna = False))

General Service               23609
International Consultant      23179
National Consultant           20823
International Professional     7953
Intern                         5275
Other                          4917
Unspecified                    3898
National Professional          3641
UNV                            1280
PSA                             720
Name: staff_category, dtype: int64

In [8]:
df_jobs['languages_required'] = df_jobs['languages_required'].apply(clean.clean_languages)
df_jobs['languages_required'].str.split(' ').explode().value_counts()

English        61217
English,       22570
French         10836
Spanish         6932
Arabic          6343
Russian         4882
Unspecified     4373
French,         1369
Chinese          712
Spanish,          68
Arabic,           30
Russian,          25
Name: languages_required, dtype: int64

In [9]:
to_drop = ['location', 'time_left', 'starting_date', 'duration_contract', 'duration_assignment']

print(f'Shape before: {df_jobs.shape}')
df_jobs.drop(to_drop, axis = 1, inplace = True)
df_jobs['year'] = df_jobs['year'].astype(int)

print(f'Shape after : {df_jobs.shape}')
display(df_jobs.head())

Shape before: (95295, 20)
Shape after : (95295, 15)


,job_id,content,title,country,region,year,type_of_contract,post_level,languages_required,staff_category,background,duties_responsibilities,competencies,skills_experiences,language
0,JOB20338108876596148de9fba9a926ad06,"['hiv and sti clinical consultant (ic)', 'loca...",hiv and sti clinical consultant (ic),FJ,Australia and Oceania,2022,Individual Contract,International Consultant,English,International Consultant,the united nations development programme (undp...,project description and consultancy rationale ...,strong interpersonal and communication skills;...,educational qualifications : minimum master s ...,en
1,JOB579f6f45d13396afc130172715c7bba3,['internship- pacific digital economy programm...,internship- pacific digital economy programme ...,SB,Australia and Oceania,2022,Internship,Intern,English,Intern,the united nations capital development fund (u...,under the guidance and supervision of unc df s...,uncdf/undp core competencies : communication d...,education : candidate must be enrolled in a de...,en
2,JOB2b8aa5ccf8370e1acb7ba55cdc1e27d9,['consultant international spécialisé dans le ...,consultant international spécialisé dans le co...,DJ,Sub-Saharian Africa,2022,Individual Contract,International Consultant,French,International Consultant,a vis de r ecru te ment d un consultant in div...,2. description du pro jet le pro jet d ap pui ...,4. liv rables attenduslivrables/résultatsdurée...,7. qualifications et experiences requisesi. qu...,fr
3,JOB6c058f6289a3634356df9788d72fb6a8,['consultant national pour l’élaboration du pl...,consultant national pour l’élaboration du plan...,DJ,Sub-Saharian Africa,2022,Individual Contract,National Consultant,French,National Consultant,a vis de r ecru te ment d un consultant in div...,3. object if s l object if principal de la mis...,9. qualification le ou la consultant do it pos...,10. term es de pai e men tle consultant sera p...,fr
4,JOB72bdf033a137fb5a2dd18ad4b6941cdd,['individual consultant - national project off...,individual consultant - national project officer,SA,Western Asia,2022,Individual Contract,National Consultant,"English, Arabic",National Consultant,post title : national project officer starting...,scope of work : ensure effective and eff...,competencies : corporate competencies : demons...,required skills and experience : educat...,en


In [11]:
df_jobs.to_parquet('../data/undp_jobs_processed.parquet')
print('saved!!!')

saved!!!
